### CPU 환경

In [1]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8153 CPU @ 2.00GHz
Stepping:                        4
CPU MHz:                         1000.091
CPU max MHz:                     2800.0000
CPU min MHz:                     1000.0000
BogoMIPS:                        4000.00
Virtualization:                  VT-x
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        32 MiB
L3 cache:                   

### GPU 환경

In [2]:
!nvidia-smi

Tue Feb 23 12:38:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:37:00.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:86:00.0 Off |                    0 |
| N/A   

### RAM 용량 확인

In [3]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:          125Gi       2.9Gi       4.2Gi       2.0Mi       118Gi       121Gi
Swap:         8.0Gi        18Mi       8.0Gi


### HDD 용량 확인

In [4]:
# 디스크 용량 확인
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             63G     0   63G   0% /dev
tmpfs            13G  2.7M   13G   1% /run
/dev/sda2       549G  385G  137G  74% /
tmpfs            63G     0   63G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            63G     0   63G   0% /sys/fs/cgroup
/dev/sda1       511M  7.8M  504M   2% /boot/efi
/dev/loop4       56M   56M     0 100% /snap/core18/1944
tmpfs            13G     0   13G   0% /run/user/1000
/dev/loop6       32M   32M     0 100% /snap/snapd/10707
/dev/loop0       70M   70M     0 100% /snap/lxd/19032
/dev/loop1       70M   70M     0 100% /snap/lxd/19188
/dev/sdb2       466G  297G  170G  64% /home/test/mount_folder
/dev/loop5       56M   56M     0 100% /snap/core18/1988
/dev/loop2       32M   32M     0 100% /snap/snapd/11036


### OS 환경

In [5]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="20.04.1 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.1 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


## 패키지 불러오기

In [6]:
import os
import os.path as osp
import sys
from glob import glob

import datetime
import time
import pytz

import pandas as pd
import numpy as np
from tqdm import tqdm

import sklearn

import torch
import torchvision
from torchinfo import summary

import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
import torchvision.models as models

import explain

In [7]:
print('python version:',sys.version)
print('pandas version:',pd.__version__)
print('numpy version:', np.__version__)

print('sklearn version:', sklearn.__version__)
print('torch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)

python version: 3.7.0 (default, Oct  9 2018, 10:31:47) 
[GCC 7.3.0]
pandas version: 1.1.5
numpy version: 1.19.2
sklearn version: 0.23.2
torch version: 1.7.1
torchvision version: 0.8.2


## 모델 파라미터 (Parameter)

In [8]:
# 모델에 입력할 Input 이미지의 크기를 지정함.
IMG_WIDTH = 512
IMG_HEIGHT = 512

# 배치 사이즈 임의 지정
batch_size = 4

# 데이터를 읽어올 경로를 지정함.
data_dir='../Dataset/preprocessed/classification'
# 학습된 모델이 저장된 경로를 지정함.
ckpt_dir = "./Models/"

# 모델 이름
Model_name = "cow_classification"
# 모델을 연산할 장비 설정 (Docker환경에서 작동할 수 있도록 CPU에서 계산함)
device = torch.device('cpu')

## Train, Validation, Test 데이터 셋 확인

In [9]:
label=['1++','1+','1','2','3']

for set_ in ['train','val','test']:
    set_size=len(glob(osp.join(data_dir, set_, '*', '*'), recursive=True))
    if set_=='val':
        print(f"{set_+'idation'} set size: {set_size}\n")
    else:
        print(f"{set_} set size: {set_size}\n")
    
    for i in range(5):
        size=len(glob(osp.join(data_dir, set_, str(i), '*'), recursive=True))
        print(f"The number of Class {label[i]}: {size} / {set_size} = {size/set_size :.2f}")
    print('-'*45)
    print()

train set size: 19520

The number of Class 1++: 4762 / 19520 = 0.24
The number of Class 1+: 5004 / 19520 = 0.26
The number of Class 1: 5940 / 19520 = 0.30
The number of Class 2: 2720 / 19520 = 0.14
The number of Class 3: 1094 / 19520 = 0.06
---------------------------------------------

validation set size: 2438

The number of Class 1++: 594 / 2438 = 0.24
The number of Class 1+: 625 / 2438 = 0.26
The number of Class 1: 742 / 2438 = 0.30
The number of Class 2: 340 / 2438 = 0.14
The number of Class 3: 137 / 2438 = 0.06
---------------------------------------------

test set size: 2440

The number of Class 1++: 595 / 2440 = 0.24
The number of Class 1+: 625 / 2440 = 0.26
The number of Class 1: 743 / 2440 = 0.30
The number of Class 2: 340 / 2440 = 0.14
The number of Class 3: 137 / 2440 = 0.06
---------------------------------------------



## 데이터 셋 (Dataset) 불러오기

In [10]:
# 이미지 transformation
test_compose=transforms.Compose([
    transforms.Resize((IMG_HEIGHT,IMG_WIDTH)),
    transforms.ToTensor()
])

# Test set을 읽어옴
test_dataset=datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_compose)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)

### 모델 불러오기 (Load)

In [11]:
# Model: ResNeXt50_32x4d
model=models.resnext50_32x4d(pretrained=False)
# 최종 Label 의 개수 = 5 (1++, 1+, 1, 2, 3)
model.fc=nn.Linear(model.fc.in_features, 5)

# 학습된 모델 가중치 불러오기
checkpoint=torch.load(ckpt_dir + Model_name + '.pth', map_location=device)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

### 모델 학습 전, ImageNet 데이터로 Pretrain 된 기본 모델 불러오기 (Load)

In [12]:
# model : ResNeXt50_32x4d
base_model=models.resnext50_32x4d(pretrained=True)
base_model.fc=nn.Linear(base_model.fc.in_features, 5)

## 모델 확인 (Summary)

In [13]:
_ = summary(model, (4, 3, 512, 512))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [4, 64, 256, 256]         9,408
├─BatchNorm2d: 1-2                       [4, 64, 256, 256]         128
├─ReLU: 1-3                              [4, 64, 256, 256]         --
├─MaxPool2d: 1-4                         [4, 64, 128, 128]         --
├─Sequential: 1-5                        [4, 256, 128, 128]        --
|    └─Bottleneck: 2-1                   [4, 256, 128, 128]        --
|    |    └─Conv2d: 3-1                  [4, 128, 128, 128]        8,192
|    |    └─BatchNorm2d: 3-2             [4, 128, 128, 128]        256
|    |    └─ReLU: 3-3                    [4, 128, 128, 128]        --
|    |    └─Conv2d: 3-4                  [4, 128, 128, 128]        4,608
|    |    └─BatchNorm2d: 3-5             [4, 128, 128, 128]        256
|    |    └─ReLU: 3-6                    [4, 128, 128, 128]        --
|    |    └─Conv2d: 3-7                  [4, 256, 128, 128]        32,768

---

## 테스트 (Test) 수행

### 1. ImageNet 데이터로 Pretrain 된 기본 모델 성능 평가

### 테스트 이미지 평가 (Predict)

In [14]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2021-02-15 15:33:30


In [15]:
since=time.time()

In [16]:
tot_labels=[]  # 전체 정답 Label 을 저장할 리스트
tot_pred_labels=[]  # 전체 예측 Label 을 저장할 리스트

# 모델을 device에 올림
base_model=base_model.to(device)
# 네트워크를 evaluation 용으로 선언
base_model=base_model.eval()

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):

        images=images.to(device)
        labels=labels.to(device)

        # Model Prediction (forward)
        output=base_model(images)
        _, output_index = torch.max(output, 1)    

        tot_labels.extend(list(labels.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).numpy()))

100%|██████████| 610/610 [17:22<00:00,  1.71s/it]


In [17]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 17m 22s


In [18]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2021-02-15 15:50:52


## Accuracy

In [19]:
result=explain.ShowResult(tot_labels, tot_pred_labels)

In [20]:
result.show_result()

Predicted  1++     1   All
True                      
1++         41   554   595
1+          22   603   625
1           28   715   743
2            8   332   340
3           14   123   137
All        113  2327  2440


#-- Confusion Matrix for class 1++

                Pred    
             Non 1++ 1++
True Non 1++    1773  72
     1++         554  41

Accuracy for class 1++ : 74.344
-----------------------------------

#-- Confusion Matrix for class 1+

              Pred   
            Non 1+ 1+
True Non 1+   1815  0
     1+        625  0

Accuracy for class 1+ : 74.385
-----------------------------------

#-- Confusion Matrix for class 1

            Pred      
           Non 1     1
True Non 1    85  1612
     1        28   715

Accuracy for class 1 : 32.787
-----------------------------------

#-- Confusion Matrix for class 2

            Pred   
           Non 2  2
True Non 2  2100  0
     2       340  0

Accuracy for class 2 : 86.066
-----------------------------------

#-- Conf

---

### 2. 기본 모델을 Fine Tuning (학습) 한 모델 성능 평가 (Transfer Learning)

### 테스트 이미지 평가 (Predict)

In [21]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2021-02-15 15:50:52


In [22]:
since=time.time()

In [23]:
tot_labels=[]  # 전체 정답 Label 을 저장할 리스트
tot_pred_labels=[]  # 전체 예측 Label 을 저장할 리스트

# 모델을 device에 올림
model=model.to(device)
# 네트워크를 evaluation 용으로 선언
model=model.eval()

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):

        images=images.to(device)
        labels=labels.to(device)

        # Model Prediction (forward)
        output=model(images)
        _, output_index = torch.max(output, 1)    

        tot_labels.extend(list(labels.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).numpy()))

100%|██████████| 610/610 [17:43<00:00,  1.74s/it]


In [24]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 17m 43s


In [25]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2021-02-15 16:08:36


## Accuracy

In [26]:
result=explain.ShowResult(tot_labels, tot_pred_labels)

In [27]:
result.show_result()

Predicted  1++   1+    1    2    3   All
True                                    
1++        577    9    8    1    0   595
1+          16  589   20    0    0   625
1            8   24  701   10    0   743
2            3    5   15  315    2   340
3            0    0    0    0  137   137
All        604  627  744  326  139  2440


#-- Confusion Matrix for class 1++

                Pred     
             Non 1++  1++
True Non 1++    1818   27
     1++          18  577

Accuracy for class 1++ : 98.156
-----------------------------------

#-- Confusion Matrix for class 1+

              Pred     
            Non 1+   1+
True Non 1+   1777   38
     1+         36  589

Accuracy for class 1+ : 96.967
-----------------------------------

#-- Confusion Matrix for class 1

            Pred     
           Non 1    1
True Non 1  1654   43
     1        42  701

Accuracy for class 1 : 96.516
-----------------------------------

#-- Confusion Matrix for class 2

            Pred     
           Non